<a href="https://colab.research.google.com/github/SihanTao/AlgoCourse/blob/main/fxPairTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import packages

In [ ]:
!pip install backtrader
!pip install quantstats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
import matplotlib.pyplot as plt
import backtrader as bt
import quantstats as qs

# Pair Trading Strategy Class

In [ ]:
class PairTrading(bt.Strategy):
    params = (
        ('window_length', 30),
        ('stock1', 's1'),
        ('stock2', 's2'),
        ('variance', 1)
    )

    def is_warm_up(self):
        return self.days_passed <= self.params.window_length

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.val_start = self.broker.get_cash()
        self.days_passed = 0
        self.value = 0

    def notify_order(self, order):
        # 1. If order is submitted/accepted, do nothing
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 2. If order is buy/sell executed, report price executed
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: {0:8.2f}, Size: {1:8.2f} Cost: {2:8.2f}, Comm: {3:8.2f}\n'.format(
                    order.executed.price,
                    order.executed.size,
                    order.executed.value,
                    order.executed.comm))
            else:
                self.log('SELL EXECUTED, {0:8.2f}, Size: {1:8.2f} Cost: {2:8.2f}, Comm{3:8.2f}\n'.format(
                    order.executed.price,
                    order.executed.size,
                    order.executed.value,
                    order.executed.comm))

            self.bar_executed = len(self) #when was trade executed
        # 3. If order is canceled/margin/rejected, report order canceled
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')


    def notify_trade(self,trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS {0:8.2f}, NET {1:8.2f}'.format(
            trade.pnl, trade.pnlcomm))

    def next(self):
        self.days_passed += 1

        if self.is_warm_up():
            return

        # self.log(f'z_score: {self.z_score[0]:.2f}, spread: {self.spread[0]:.2f}')
        self.log(f'Close: {self.datas[0][0]:.2f}, {self.datas[1][0]:.2f}')
        self.log(f'Open: {self.datas[0].open[0]:.2f}, {self.datas[1].open[0]:.2f}')
        s1_position, s2_position = self.getposition(data=self.datas[0]), self.getposition(data=self.datas[1])

        data1_series = pd.Series(self.datas[0].close.get(size=self.params.window_length))
        data2_series = pd.Series(self.datas[1].close.get(size=self.params.window_length))

        # compute the spread and z_score
        spread = data1_series - data2_series
        z_score = (spread.iloc[-1] - spread.mean()) / spread.std()

        if self.position:
            print(f'Current position:\n {s1_position}, {s2_position}')
            if -self.params.variance < z_score < self.params.variance:
                print(f'Close Position\n')
                self.close(data=self.datas[0], size=s1_position.size)
                self.close(data=self.datas[1], size=s2_position.size)

        if not self.position:
            if z_score < -self.params.variance:
                # Long the spread: long stock1 and short stock2
                size = int(self.broker.get_cash() / self.datas[0])
                self.log(
                    f'''{self.params.stock1}: Close: {self.datas[0][0]:.2f}, {self.params.stock2} Close: {self.datas[1][0]:.2f}''')
                print(f'BUY size={size}')
                self.sell(self.datas[1], size=size)
                self.buy(self.datas[0], size=size)
            elif z_score > self.params.variance:
                # Short the spread: short stock1 and long stock2
                size = int(self.broker.get_cash() / self.datas[1])
                self.log(
                    f'''{self.params.stock1}: Close: {self.datas[0][0]:.2f}, {self.params.stock2} Close: {self.datas[1][0]:.2f}''')
                print(f'SELL size={size}')
                self.sell(self.datas[0], size=size)
                self.buy(self.datas[1], size=size)

    def stop(self):
        self.value = round(self.broker.get_value(), 2)

# Download Data

In [ ]:
# Define the two stocks
investment_universe = ['GBPUSD=X', 'EURUSD=X']

# Download historical data as pandas DataFrame
start_date = '2012-01-01'
end_date = '2015-01-01'

symbol_to_df = {}
for stock in investment_universe:
    symbol_to_df[stock] = yf.download(stock, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Optimize strategy parameters

In [ ]:
import time

t_start = time.time()

cerebro = bt.Cerebro(stdstats=False)
cerebro.broker.setcash(100000.0)

for k, v in symbol_to_df.items():
    cerebro.adddata(bt.feeds.PandasData(dataname=symbol_to_df[k]))

cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')

cerebro.optstrategy(PairTrading, variance=np.arange(0.8, 2.0 + 0.01, 0.1),\
          window_length=range(5, 120+1))

# Run the strategy
opt_runs = cerebro.run()

total_runtime = time.time() - t_start
print(f"Optimization finished. Took {total_runtime:.2f}s")


2012-01-10, Close: 1.55, 1.282012-01-11, Close: 1.55, 1.28

2012-01-10, Open: 1.55, 1.282012-01-12, Close: 1.53, 1.27
2012-01-09, Close: 1.54, 1.272012-01-13, Close: 1.53, 1.28
2012-01-16, Close: 1.53, 1.262012-01-12, Open: 1.53, 1.27


2012-01-11, Open: 1.55, 1.282012-01-17, Close: 1.53, 1.272012-01-11, Close: 1.55, 1.282012-01-13, Open: 1.53, 1.282012-01-19, Close: 1.54, 1.292012-01-16, Open: 1.53, 1.26
2012-01-20, Close: 1.55, 1.30





2012-01-18, Close: 1.53, 1.272012-01-11, Open: 1.55, 1.282012-01-09, Open: 1.54, 1.272012-01-17, Open: 1.53, 1.272012-01-12, s1: Close: 1.53, s2 Close: 1.27


2012-01-20, Open: 1.55, 1.30

2012-01-19, Open: 1.54, 1.29BUY size=652052012-01-24, Close: 1.56, 1.30
2012-01-11, s1: Close: 1.55, s2 Close: 1.282012-01-18, Open: 1.53, 1.27



2012-01-13, s1: Close: 1.53, s2 Close: 1.28
2012-01-17, Close: 1.53, 1.27
2012-01-09, s1: Close: 1.54, s2 Close: 1.27
2012-01-23, Close: 1.55, 1.292012-01-24, Open: 1.56, 1.302012-01-19, s1: Close: 1.54, s2 Close: 1.29



In [ ]:
results = []
for run in opt_runs:
    for res in run:
      variance, window_length = res.p.variance, res.p.window_length
      returns = res.analyzers.returns.get_analysis()


In [ ]:
returns, positions, transactions, gross_lev = opt_runs[0][0].analyzers.pyfolio.get_pf_items()

In [ ]:
returns

index
2012-01-02 00:00:00+00:00    0.000000
2012-01-03 00:00:00+00:00    0.000000
2012-01-04 00:00:00+00:00    0.000000
2012-01-05 00:00:00+00:00    0.000000
2012-01-06 00:00:00+00:00    0.000000
                               ...   
2014-12-25 00:00:00+00:00   -0.001316
2014-12-26 00:00:00+00:00    0.000930
2014-12-29 00:00:00+00:00   -0.002600
2014-12-30 00:00:00+00:00    0.001761
2014-12-31 00:00:00+00:00   -0.003614
Name: return, Length: 782, dtype: float64

In [ ]:
total_return = np.prod(1 + returns) - 1
total_return

0.11382852066278248

In [ ]:
return_ana = opt_runs[0][0].analyzers.returns.get_analysis()

In [ ]:
return_ana

OrderedDict([('rtot', 0.10780319847405222),
             ('ravg', 0.00013785575252436345),
             ('rnorm', 0.035350119926919596),
             ('rnorm100', 3.5350119926919596)])

In [ ]:
opt_runs[0][0].p.window_length

30

In [ ]:
print(len(opt_runs))
print(len(opt_runs[0]))

1
1
